In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Read data

In [2]:
# Read QuickVina02 results
qvina         = pd.read_csv('qvina.csv')
qvina.columns = ['CID','pose','qvina']

# Read QVina rescored with RF-Score
rfscore_qvina         = pd.read_csv('rfscore_qvina.csv')
rfscore_qvina.columns = ['CID','pose','rfscore_qvina']

In [3]:
top_qvina  = pd.merge(qvina.query('pose == 1'), rfscore_qvina.query('pose == 1'))
top_qvina.drop('pose', axis=1, inplace=True)
top_qvina.head()

,CID,qvina,rfscore_qvina
0,MAR-UNI-c84db004-13,-9.0,6.586498
1,BAR-COM-4e090d3a-53,-7.5,7.221692
2,TRY-UNI-714a760b-6,-6.4,6.377253
3,EDJ-MED-c9f55a56-1,-6.2,5.491030
4,LON-WEI-0a73fcb8-7,-7.4,6.677942


In [4]:
## Sort Qvina rescored with RF-Score
#top_rfscore_qvina = rfscore_qvina.sort_values('rfscore_qvina', ascending=False).groupby('CID').head(1)
#pd.merge(qvina.query('pose == 1'), rfscore_qvina.query('pose == 1')).append(pd.merge(qvina, rfscore_qvina.sort_values('rfscore_qvina', ascending=False).groupby('CID').head(1), on=['CID','pose']))

In [5]:
# Read PLANTS results
plants         = pd.read_csv('plants.csv')
plants.columns = ['CID','pose','plants']

# Read PLANTS rescored with RF-Score
rfscore_plants         = pd.read_csv('rfscore_plants.csv', header=None)
rfscore_plants.columns = ['rfscore_plants','CID']
rfscore_plants         = rfscore_plants[['CID','rfscore_plants']]
rfscore_plants[['CID','pose']] = rfscore_plants['CID'].str.split('_', expand=True)
rfscore_plants['pose']         = rfscore_plants['pose'].astype('int')

In [6]:
top_plants  = pd.merge(plants.query('pose == 1'), rfscore_plants.query('pose == 1'))
top_plants.drop('pose', axis=1, inplace=True)
top_plants.head()

,CID,plants,rfscore_plants
0,EDG-MED-0da5ad92-3,-70.8540,6.885798
1,DAR-DIA-23aa0b97-19,-79.4982,6.852397
2,MAT-POS-590ac91e-22,-70.4662,6.217164
3,LON-WEI-1908424e-3,-90.5268,7.534084
4,GIA-UNK-30c7cb75-1,-76.6202,6.839672


In [7]:
# Sort PLANTS rescored with RF-Score
#top_rfscore_plants = rfscore_plants.sort_values('rfscore_plants', ascending=False).groupby('CID').head(1)

In [8]:
# Read experimental data
exp = pd.read_csv('activity_data.csv')

In [9]:
exp.head()

,SMILES,CID,canonical_CID,r_inhibition_at_20_uM,r_inhibition_at_50_uM,r_avg_IC50,f_inhibition_at_20_uM,f_inhibition_at_50_uM,f_avg_IC50,f_avg_pIC50,relative_solubility_at_20_uM,relative_solubility_at_100_uM,trypsin_IC50,NMR_std_ratio,acrylamide,chloroacetamide,series
0,COC(=O)C(C(=O)Nc1cnccc1C)c1cccc(Cl)c1,MAT-POS-1e5f28a7-1,MAT-POS-1e5f28a7-1,NaN,NaN,17.091397,NaN,NaN,23.712295,4.625026,NaN,NaN,NaN,NaN,False,False,3-aminopyridine-like
1,OCCn1c(CSc2nc3ccccc3o2)nc2ccc(Cl)cc21,MAT-POS-e10a589d-1,MAT-POS-e10a589d-1,NaN,NaN,99.000000,NaN,NaN,0.326703,6.485847,NaN,NaN,NaN,NaN,False,False,NaN
2,N#Cc1ncn(CC(=O)Nc2ccc(Br)cc2Cl)n1,MAT-POS-e10a589d-4,MAT-POS-e10a589d-4,NaN,NaN,NaN,NaN,NaN,15.730897,4.803247,NaN,NaN,NaN,NaN,False,False,3-aminopyridine-like
3,Cn1nc(-c2ccc(C(F)(F)F)cc2)nc2c(=O)n(C)c(=O)nc1-2,MAT-POS-1e0c1c67-1,MAT-POS-1e0c1c67-1,NaN,NaN,99.000000,NaN,NaN,2.400602,5.619680,NaN,NaN,NaN,NaN,False,False,NaN
4,Cc1cc2c(cc1C)C(C(=O)N1CCCCC1c1cn[nH]c1)CO2,MAT-POS-6da3605a-1,MAT-POS-6da3605a-1,NaN,NaN,99.000000,NaN,NaN,99.500000,NaN,NaN,NaN,NaN,NaN,False,False,NaN


In [10]:
tmp  = pd.merge(top_qvina, top_plants)
data = pd.merge(tmp, exp)

In [11]:
# Normalize scores
data['plants'] /= 10
data[['rfscore_qvina','rfscore_plants']] *= -1

In [12]:
import pingouin as pg

In [13]:
for SF in ['qvina','rfscore_qvina','plants','rfscore_plants']:
    df = data[(data['r_avg_IC50'] > 0) & (data['r_avg_IC50'] < 10) & (data[SF] < 0)]
    print(f'{SF}: {pg.corr(df[SF], df["r_avg_IC50"], method="shepherd")["r2"].item():.4f}')

qvina: 0.0323
rfscore_qvina: 0.0115
plants: 0.0859
rfscore_plants: 0.0667


In [14]:
# Sem remover outliers
for SF in ['qvina','rfscore_qvina','plants','rfscore_plants']:
    print(f'{SF}: {pg.corr(data[SF], data["r_avg_IC50"], method="shepherd")["r2"].item():.4f}')

qvina: 0.0339
rfscore_qvina: 0.0315
plants: 0.0048
rfscore_plants: 0.0340


In [7]:
top = data.query('pose == 1')

In [8]:
descriptors = pd.read_csv('descriptors.csv')

In [11]:
descriptors.drop('Reference', axis=1, inplace=True)

In [16]:
top.query('PUBCHEM_ACTIVITY_OUTCOME == "Active"')

,PUBCHEM_SID,PUBCHEM_ACTIVITY_OUTCOME,Inhibition,pose,vina_score,smina_score
2010,842353,Active,21.34,1,-6.2,-6.54804
2310,842393,Active,12.30,1,-6.8,-6.90986
8370,843085,Active,57.07,1,-5.8,-6.08944
9450,843208,Active,25.40,1,-6.6,-6.58671
26948,845167,Active,16.79,1,-6.1,-6.15173
...,...,...,...,...,...,...
2707266,50086403,Active,13.62,1,-6.5,-6.67264
2708476,50086550,Active,12.37,1,-6.0,-6.12802
2714964,51086478,Active,12.87,1,-7.1,-7.23337
2721052,51090130,Active,12.59,1,-6.3,-6.32793


In [17]:
descriptors

,CID,r_inhibition_at_50_uM,f_inhibition_at_50_uM,Tanimoto,VABC Volume Descriptor,Rotatable Bonds Count,Topological Polar Surface Area,Molecular Weight,XLogP
0,MAT-POS-1e5f28a7-1,NaN,NaN,0.367647,281.26289399511734,8,68.29,318.077120,3.070
1,MAT-POS-e10a589d-1,NaN,NaN,0.322581,272.7615187845899,7,89.38,359.049525,2.926
2,MAT-POS-e10a589d-4,NaN,NaN,0.333333,229.47641706672832,5,83.60,338.952250,1.945
3,MAT-POS-1e0c1c67-1,NaN,NaN,0.333333,272.4061287470087,7,82.67,337.078659,4.085
4,MAT-POS-6da3605a-1,NaN,NaN,0.315789,300.70631360989046,4,58.22,325.179027,3.153
...,...,...,...,...,...,...,...,...,...
999,DAR-DIA-842b4336-4,35.740000,23.748390,0.408163,194.68523450616138,3,82.26,253.007661,0.906
1000,DAR-DIA-842b4336-7,8.515000,8.855559,0.448980,180.75212958685586,3,80.05,218.080376,0.102
1001,DAR-DIA-842b4336-13,NaN,11.500410,0.420000,200.9844589238615,4,70.23,252.012412,0.862
1002,DAR-DIA-842b4336-16,0.000000,-9.906875,0.440000,187.05135400455592,4,68.02,217.085127,-0.371


# Split train and test variables

In [8]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics

In [9]:
X = top[['vina_score','smina_score']].values
y = pd.Categorical(top['PUBCHEM_ACTIVITY_OUTCOME']).codes # 1 para inativo, 0 para ativo

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

# Dummy classifier

In [11]:
from sklearn.dummy import DummyClassifier

In [12]:
dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
dummy_pred = dummy.predict(X_test)

In [13]:
# checking unique labels
print('Unique predicted labels: ', (np.unique(dummy_pred)))

# checking accuracy
print('Test score: ', metrics.accuracy_score(y_test, dummy_pred))

Unique predicted labels:  [1]
Test score:  0.9985315280918676


In [14]:
def print_metrics(y_test, y_pred):
    print(f'accuracy_score: {metrics.accuracy_score(y_test, y_pred)}')
    print(f'f1_score: {metrics.f1_score(y_test, y_pred)}')
    print(f'recall_score: {metrics.recall_score(y_test, y_pred)}')

# Logistic regression

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)

In [17]:
lr_pred = lr.predict(X_test)

In [18]:
predictions = pd.DataFrame(lr_pred)
predictions[0].value_counts()

1    68098
Name: 0, dtype: int64

In [19]:
print_metrics(y_test, lr_pred)

accuracy_score: 0.9985315280918676
f1_score: 0.9992652245473783
recall_score: 1.0


# Random forest

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

In [22]:
rfc_pred = rfc.predict(X_test)

In [23]:
print_metrics(y_test, rfc_pred)

accuracy_score: 0.9976210755088255
f1_score: 0.9988091212490995
recall_score: 0.9990882084767199


In [24]:
def predict(resampled):
    y_train = resampled['PUBCHEM_ACTIVITY_OUTCOME']
    X_train = resampled.drop('PUBCHEM_ACTIVITY_OUTCOME', axis=1)
    
    resampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)
    resampled_pred = resampled.predict(X_test)
    
    return resampled_pred

# Oversample minority class

In [25]:
from sklearn.utils import resample

In [26]:
X = pd.concat([pd.DataFrame(X_train, columns=['vina_score','smina_score']), pd.Series(y_train, name='PUBCHEM_ACTIVITY_OUTCOME')], axis=1)

In [27]:
inactive = X.query('PUBCHEM_ACTIVITY_OUTCOME == 1')
active = X.query('PUBCHEM_ACTIVITY_OUTCOME == 0')

In [28]:
new_active = resample(active, replace=True, n_samples=len(inactive), random_state=27) 

In [29]:
upsampled = pd.concat([inactive, new_active])

In [30]:
upsampled['PUBCHEM_ACTIVITY_OUTCOME'].value_counts()

1    204007
0    204007
Name: PUBCHEM_ACTIVITY_OUTCOME, dtype: int64

In [31]:
upsampled_pred = predict(upsampled)
print_metrics(y_test, upsampled_pred)

accuracy_score: 0.5353754882669095
f1_score: 0.6970857427335044
recall_score: 0.535398099944116


# Undersample majority class

In [32]:
new_inactive = resample(inactive, replace=False, n_samples=len(active), random_state=27)

In [33]:
downsampled = pd.concat([new_inactive, active])

In [34]:
downsampled['PUBCHEM_ACTIVITY_OUTCOME'].value_counts()

1    285
0    285
Name: PUBCHEM_ACTIVITY_OUTCOME, dtype: int64

In [35]:
downsampled_pred = predict(downsampled)
print_metrics(y_test, downsampled_pred)

accuracy_score: 0.5218361772739287
f1_score: 0.6854581634821585
recall_score: 0.521780052354481


# SMOTE (Synthetic Minority Oversampling Technique)

In [36]:
from imblearn.over_sampling import SMOTE

In [37]:
sm = SMOTE(random_state=27)
X_train, y_train = sm.fit_sample(X_train, y_train)

In [38]:
smote = LogisticRegression(solver='liblinear').fit(X_train, y_train)

In [39]:
smote_pred = smote.predict(X_test)
print_metrics(y_test, smote_pred)

accuracy_score: 0.5380774765778731
f1_score: 0.6993826334601196
recall_score: 0.5381187681990647
